In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from sklearn.metrics import roc_auc_score

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
train.shape

(200000, 202)

In [5]:
train.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [6]:
print(f'Percentage of label 0 samples = {179902/2000}')
print(f'Percentage of label 1 samples = {20098/2000}')

Percentage of label 0 samples = 89.951
Percentage of label 1 samples = 10.049


In [7]:
train[train['target']==0].describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,179902.0,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.00000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,...,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000,179902.000000
mean,0.0,10.626681,-1.695770,10.665876,6.788979,11.072412,-5.146736,5.389620,16.549306,0.262347,7.584907,0.398410,-3.291479,14.028392,8.615764,7.542364,14.570748,9.326326,-5.698669,15.129914,12.408389,13.326894,17.418103,4.247785,3.024026,10.548447,13.666224,-4.178694,-1.137611,5.539248,5.04978,-7.689438,10.411541,-0.539278,14.838196,11.443788,3.779194,2.227270,5.863121,10.640744,...,24.277974,5.632989,5.347406,10.946221,-2.946693,19.409346,2.970474,-4.231423,4.922471,5.641983,-0.075908,-0.858512,19.927197,-0.761772,20.357774,11.619644,-2.818344,11.915324,-0.957623,2.544666,-2.796515,10.079230,0.740672,8.777265,12.606196,-3.983345,9.002707,-10.392583,15.422021,0.743065,3.149130,7.390800,1.949017,3.355403,18.017716,-0.155601,2.260297,8.919032,15.924058,-3.415273
std,0.0,3.008564,4.024813,2.612961,2.040082,1.620103,7.827522,0.857983,3.417700,3.331105,1.228839,5.499197,5.955722,0.187441,4.603038,2.248042,0.410909,2.551915,6.716530,7.805564,7.981264,5.871569,8.137105,2.816560,0.525768,3.768895,0.284984,5.847831,1.522704,0.780864,2.61397,7.967595,2.159704,2.579291,4.302266,0.538577,5.158947,3.107991,2.246682,4.276824,...,10.875049,0.217820,1.412943,5.226997,5.430098,4.983414,0.366132,7.767637,3.094492,0.367496,4.387806,5.363749,8.633896,5.942170,7.064789,2.883026,7.505930,2.619443,8.565845,2.766575,5.232564,1.368631,8.961104,4.470965,9.262205,4.725591,3.182983,11.561942,3.923825,0.975902,4.522568,2.997847,1.476456,3.982819,3.127715,1.422275,5.441118,0.917467,2.978539,10.434525
min,0.0,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,-20.731300,-26.095000,13.434600,-6.011100,1.169000,13.076900,0.635100,-33.380200,-10.664200,-11.557600,-5.432200,-10.089000,-5.164000,1.209800,-0.678400,12.720000,-24.243100,-6.166800,2.089600,-4.78720,-34.798400,2.140600,-8.986100,1.508500,9.816900,-16.513600,-8.095100,-1.183400,-5.908200,...,-7.452200,4.852600,0.623100,-6.531700,-19.997700,3.816700,1.851200,-35.969500,-5.250200,4.258800,-14.506000,-22.479300,-11.453300,-22.748700,-2.995300,3.241500,-29.116500,4.952100,-29.273400,-7.856100,-22.037400,5.416500,-26.001100,-4.676100,-18.489700,-22.583300,-3.022300,-46.598100,4.412300,-2.554300,-14.093300,-2.691700,-3.814500,-10.845500,8.694400,-5.261000,-14.209600,5.960600

In [8]:
train[train['target']==1].describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,20098.0,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,...,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000
mean,1.0,11.156418,-1.017613,11.156633,6.864113,11.131337,-4.336522,5.581966,16.514917,0.479432,7.409064,0.357916,-2.834888,13.984464,7.764618,7.495020,14.594414,9.395369,-5.679385,16.265343,12.708704,12.968654,15.823715,4.821429,2.979384,10.906222,13.678882,-2.949112,-1.140560,5.476866,5.090517,-7.672537,10.227496,-0.276639,14.200831,11.348866,4.409162,1.828816,5.920623,10.654546,...,24.092142,5.636010,5.501557,11.502983,-2.202475,18.477984,2.899436,-3.432649,5.068286,5.582531,0.630093,-0.592465,18.831534,0.072190,18.893971,11.828309,-2.631667,11.592995,-1.519278,3.010162,-2.250702,10.141805,0.526091,8.695892,14.103659,-3.982510,8.679957,-9.819989,14.975745,0.772981,3.998064,7.864560,1.738266,3.120260,17.779568,-0.021130,2.688583,8.810815,15.393283,-2.532243
std,0.0,3.270293,4.220638,2.841075,2.070898,1.649266,8.140281,0.922442,3.421365,3.340028,1.278709,5.515060,6.083536,0.207935,4.888186,2.246319,0.418237,2.605433,6.677587,8.179688,8.128723,5.908411,8.578688,3.067136,0.535207,3.836304,0.290182,6.446004,1.532760,0.803272,2.633303,7.943894,2.154552,2.651428,4.457194,0.560916,5.327264,3.203155,2.276233,4.297554,...,10.925673,0.218979,1.470759,5.541083,5.655892,5.302411,0.394379,8.029430,3.204134,0.382246,4.694040,5.498356,8.966886,6.131035,7.623027,2.966222,7.583376,2.694812,8.687718,3.041853,5.487079,1.399232,8.982184,4.509661,9.702796,4.721488,3.235419,11.676023,4.104278,0.979948,4.814830,3.210779,1.482537,4.067550,3.193153,1.485975,5.556892,0.952554,3.248123,10.435910
min,1.0,0.452800,-14.037000,2.946200,0.374000,5.876200,-28.246100,2.496000,7.302400,-9.839100,4.186100,-18.386600,-23.707100,13.499400,-4.966100,1.013300,13.287000,1.369900,-28.086100,-6.587000,-12.402500,-1.625800,-8.005400,-5.322500,1.538600,0.238200,12.782000,-23.281000,-5.599200,3.092900,-4.019700,-31.692900,2.634600,-8.084300,2.111100,9.907600,-14.591800,-7.383700,-0.585300,-6.337100,...,-4.364200,4.941800,0.623100,-5.002100,-17.835800,4.275500,1.977200,-31.216400,-4.277600,4.539500,-13.724900,-18.708600,-9.455300,-21.397200,-2.037100,3.914900,-25.254500,5.261200,-24.957600,-7.137600,-19.493600,5.637900,-23.125000,-4.808200,-15.139200,-20.223100,-2.018300,-47.753600,4.535900,-2.311700,-11.906900,-2.343000,-3.317700,-11.783400,10.120700,-5.018500,-14.020400,6.119000,6.558700,-38.852800
25%,1.0,8.695875,-4.203475,8.961125,5.314300,9.911250,-10.61580

In [9]:
test.shape

(200000, 201)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
features = train.columns.drop(['ID_code','target'])

In [12]:
X = train[features]
Y = train['target']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.27, random_state=43,stratify=Y)

In [14]:
from lightgbm import LGBMClassifier

In [15]:
lgbm = LGBMClassifier(num_leaves=2, importance_type='gain',
                              n_estimators=1000, max_bin=45,min_data_in_leaf=5,
                      learning_rate=0.235)

In [16]:
lgbm.fit(x_train,y_train,eval_metric='auc')

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='gain', learning_rate=0.235, max_bin=45,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_data_in_leaf=5, min_split_gain=0.0, n_estimators=1000,
        n_jobs=-1, num_leaves=2, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [17]:
pred_test = lgbm.predict(x_test)

In [18]:
roc_auc_score(y_test,pred_test)

0.6326149160720589

In [19]:
roc_auc_score(y_train,lgbm.predict(x_train))

0.6399661237620132

In [20]:
def leaf_estimator(n_leafs,x_train,x_test,y_train,y_test):
    auc_scores_train = []
    auc_scores_test = []
    for i in n_leafs:
        lgbm = LGBMClassifier(num_leaves=45, importance_type='gain',
                              n_estimators=1200, max_bin=i,min_data_in_leaf=100,learning_rate=0.235)
        lgbm.fit(x_train,y_train,eval_metric='auc')
        print(i,"->",roc_auc_score(y_train,lgbm.predict(x_train)),"(train auc)",
              "->",roc_auc_score(y_test,lgbm.predict(x_test)),"(test auc)")
        auc_scores_train.append(roc_auc_score(y_train,lgbm.predict(x_train)))
        auc_scores_test.append(roc_auc_score(y_test,lgbm.predict(x_test)))
    return (auc_scores_train,auc_scores_test)

In [21]:
#n_leafs = [int(x) for x in np.linspace(start =105, stop = 1000, num = 100)]
#auc_train,auc_test = leaf_estimator(n_leafs)
#plt.plot(n_leafs,auc_train,'b',label='auc train')
#plt.plot(n_leafs,auc_test,'r', label='auc test')

In [22]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 1000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 5, stop = 120, num = 10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,15,20,25,30,35]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,8,16,32,64]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 125, 135, 145, 155, 165, 175, 185, 195, 206, 216, 226, 236, 246, 256, 266, 276, 286, 296, 306, 316, 326, 336, 346, 356, 366, 376, 386, 396, 407, 417, 427, 437, 447, 457, 467, 477, 487, 497, 507, 517, 527, 537, 547, 557, 567, 577, 587, 597, 608, 618, 628, 638, 648, 658, 668, 678, 688, 698, 708, 718, 728, 738, 748, 758, 768, 778, 788, 798, 809, 819, 829, 839, 849, 859, 869, 879, 889, 899, 909, 919, 929, 939, 949, 959, 969, 979, 989, 1000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [5, 17, 30, 43, 56, 68, 81, 94, 107, 120], 'min_samples_split': [2, 5, 10, 15, 20, 25, 30, 35], 'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64], 'bootstrap': [True, False]}


In [23]:
'''rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)'''

'rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, \n                               n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)'

In [24]:
#rf_random.fit(x_train,y_train)

In [25]:
sample = pd.read_csv('../input/sample_submission.csv')

In [26]:
sample.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [27]:
test['target'] = lgbm.predict(test[features])

In [28]:
test[['ID_code','target']].to_csv('my_sub.csv',index=False)

lets try PCA and train our model by reducing some dimensions and extracting features with high variance

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
sc = StandardScaler()
x_scaled_train = sc.fit_transform(X)
x_scaled_test = sc.transform(test[features])

In [31]:
from sklearn.decomposition import PCA

In [32]:
pca = PCA(n_components=None)
pca_xtrain = pca.fit_transform(x_scaled_train)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

[0.00611308 0.00530267 0.00529862 0.00529673 0.00528794 0.00528269
 0.00527948 0.00527426 0.00526566 0.00526283 0.00525555 0.00524994
 0.00524602 0.00523687 0.00523581 0.00523504 0.00522727 0.00522382
 0.00521927 0.00521826 0.00521497 0.00521018 0.00520729 0.00520242
 0.00519975 0.0051945  0.00519299 0.00519256 0.00518746 0.00518349
 0.00518289 0.0051783  0.00517394 0.00516921 0.00516632 0.00516597
 0.00516242 0.00515977 0.00515468 0.00515336 0.00514949 0.00514498
 0.00514259 0.00514111 0.00513703 0.00513446 0.00513183 0.00512903
 0.00512785 0.00512514 0.00511907 0.00511775 0.00511637 0.00511503
 0.00510941 0.00510631 0.00510588 0.00510316 0.00510131 0.00509822
 0.00509533 0.00509268 0.00509112 0.005088   0.00508395 0.0050805
 0.0050792  0.00507578 0.00507203 0.00506889 0.00506791 0.00506343
 0.00506223 0.00505915 0.00505638 0.00505449 0.00505207 0.00505069
 0.00504641 0.00504345 0.00504256 0.00503969 0.00503893 0.00503396
 0.00503032 0.00502848 0.00502696 0.00502398 0.00502132 0.00501

In [33]:
len(explained_variance)

200

In [34]:
np.sum(explained_variance[0:171])

0.8619849736173372

This tells out of all 200 features 140 tell almost 71% variance of the data.
Lets train our lgbm model on these 140 pca features and check if accuracy improves

In [35]:
my_pca = PCA(n_components=170)
pca_xtrain = my_pca.fit_transform(x_scaled_train)

In [36]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(pca_xtrain, Y, test_size=0.27, 
                                                        random_state=43,stratify=Y)

In [37]:
lgbm2 = LGBMClassifier(num_leaves=3, importance_type='gain',
                              n_estimators=1000, max_bin=45,min_data_in_leaf=5,
                       learning_rate=0.235)

In [38]:
lgbm2.fit(x_train2,y_train2,eval_metric='auc')

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='gain', learning_rate=0.235, max_bin=45,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_data_in_leaf=5, min_split_gain=0.0, n_estimators=1000,
        n_jobs=-1, num_leaves=3, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [39]:
roc_auc_score(y_train2,lgbm2.predict(x_train2))

0.6636587766928453

In [40]:
roc_auc_score(y_test2,lgbm2.predict(x_test2))

0.6383985949383305

In [41]:
#n_leafs = [int(x) for x in np.linspace(start =40, stop = 1000, num = 50)]
#auc_train,auc_test = leaf_estimator(n_leafs,x_train2, x_test2, y_train2, y_test2)

> The target values are not equally sampled,
let try data-augmentation technique of undersampling and
check if accuracy improves

In [42]:
count_class_0, count_class_1 = train.target.value_counts()

#division of dataframe by classes
df_class_0 = train[train['target']==0]
df_class_1 = train[train['target']==1]

In [43]:
df_class_0_under = df_class_0.sample(n=count_class_1,random_state=1)

In [44]:
df_class_0_under.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,20098.0,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,...,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000
mean,0.0,10.616324,-1.695526,10.627378,6.781080,11.061589,-5.153414,5.384193,16.554337,0.301068,7.578690,0.354284,-3.302023,14.028308,8.613943,7.554553,14.572813,9.324703,-5.733770,15.094531,12.405354,13.363364,17.387838,4.235824,3.019302,10.535146,13.667841,-4.185528,-1.143422,5.540955,5.012003,-7.706978,10.387021,-0.553457,14.829125,11.440306,3.845833,2.228263,5.868273,10.677122,...,24.208852,5.632888,5.332553,10.956077,-2.948932,19.463479,2.969137,-4.257316,4.913371,5.643513,-0.098925,-0.902787,19.941256,-0.788037,20.364623,11.642322,-2.807595,11.928298,-0.934266,2.543411,-2.719781,10.068937,0.750220,8.790271,12.563008,-4.009268,9.001730,-10.452398,15.443155,0.751582,3.135637,7.433395,1.974896,3.327077,18.017576,-0.162864,2.218745,8.919340,15.887446,-3.294129
std,0.0,2.996568,4.003941,2.615047,2.019663,1.620141,7.802350,0.857797,3.413109,3.340432,1.235006,5.465689,5.941778,0.186511,4.583635,2.246801,0.411549,2.531910,6.719052,7.818226,7.996003,5.852266,8.129993,2.811577,0.527970,3.753240,0.284180,5.851307,1.520977,0.779866,2.622943,7.991004,2.160639,2.590020,4.344629,0.538761,5.194583,3.089101,2.245072,4.302873,...,10.821292,0.218997,1.414867,5.228946,5.442011,4.998581,0.365728,7.759787,3.087760,0.366927,4.389840,5.351379,8.600370,5.937507,7.111418,2.900560,7.482558,2.598400,8.525569,2.773569,5.246741,1.363783,8.944726,4.459870,9.290217,4.728001,3.172415,11.646185,3.924752,0.976710,4.534997,3.020896,1.481632,4.001098,3.138438,1.424198,5.435898,0.917691,2.997725,10.473950
min,0.0,1.186200,-13.848500,2.789200,0.043000,5.562000,-29.188300,2.385700,5.349700,-10.505500,4.197800,-18.479500,-22.369900,13.533300,-4.068200,1.353200,13.322200,0.692800,-28.333100,-7.903300,-9.715600,-2.304200,-8.841500,-5.079900,1.583100,-0.219500,12.799500,-23.854500,-5.438400,3.246500,-3.929200,-32.598500,2.503100,-8.281800,1.865500,9.892000,-15.015200,-7.791800,-0.395800,-2.766800,...,-3.677000,4.941500,0.902700,-5.051300,-19.059600,4.627800,2.013600,-35.223300,-5.221800,4.549800,-14.160500,-19.591900,-8.545600,-19.901600,-1.930500,3.889500,-25.511200,4.952100,-28.435100,-7.856100,-19.368700,5.681800,-25.272600,-4.390300,-15.060100,-20.294100,-1.157100,-42.665900,5.738800,-2.136800,-13.454700,-2.009400,-3.515900,-10.284600,9.445400,-4.607600,-13.030700,6.265600,7.070900,-37.223000
25%,0.0,8.430475,-4.758075,8.671725,5.273000,9.869225,-11.22

In [45]:
df_train_under = pd.concat([df_class_0_under,df_class_1],axis=0)

In [46]:
df_train_under.target.value_counts()

1    20098
0    20098
Name: target, dtype: int64

In [47]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(df_train_under[features], df_train_under['target'], test_size=0.27, 
                                                        random_state=40,stratify=df_train_under['target'])

In [48]:
print(f'train count = {y_train3.value_counts()}')
print(f'test count = {y_test3.value_counts()}')

train count = 1    14672
0    14671
Name: target, dtype: int64
test count = 0    5427
1    5426
Name: target, dtype: int64


In [49]:
lgbm3 = LGBMClassifier(num_leaves=10, importance_type='gain',
                              n_estimators=1200, min_data_in_leaf=90,
                       learning_rate=0.008,bagging_freq= 9,bagging_fraction= 0.375,feature_fraction =0.05)

In [50]:
lgbm3.fit(x_train3,y_train3,eval_metric='auc')

LGBMClassifier(bagging_fraction=0.375, bagging_freq=9, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.05,
        importance_type='gain', learning_rate=0.008, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=90,
        min_split_gain=0.0, n_estimators=1200, n_jobs=-1, num_leaves=10,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [51]:
roc_auc_score(y_train3,lgbm3.predict(x_train3))

0.8328736012639866

In [52]:
roc_auc_score(y_test3,lgbm3.predict(x_test3))

0.8015283746996543

In [53]:
test['target'] = lgbm3.predict(test[features])
test[['ID_code','target']].to_csv('my_sub3.csv',index=False)

In [54]:
'''lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}'''

'lgb_params = {\n    "objective" : "binary",\n    "metric" : "auc",\n    "boosting": \'gbdt\',\n    "max_depth" : -1,\n    "num_leaves" : 13,\n    "learning_rate" : 0.01,\n    "bagging_freq": 5,\n    "bagging_fraction" : 0.4,\n    "feature_fraction" : 0.05,\n    "min_data_in_leaf": 80,\n    "min_sum_heassian_in_leaf": 10,\n    "tree_learner": "serial",\n    "boost_from_average": "false",\n    #"lambda_l1" : 5,\n    #"lambda_l2" : 5,\n    "bagging_seed" : random_state,\n    "verbosity" : 1,\n    "seed": random_state\n}'